In [1]:
import nibabel
import nibabel.affines
from PIL import Image
import os
import fnmatch
import numpy as np
import ants
import scipy.io as sio
import pickle
import concurrent.futures
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import xmltodict

np.set_printoptions(precision=2, suppress=True)

## File handling and loading functions

In [2]:
# Load an image into nibabel
def load_image(data_path, filename):
    
    return nibabel.load(f"{data_path}/{filename}")

# Display the middle slice of a nibabel image
def display_image(image):

    # Get image data array from image object
    image_array = np.asarray(image.dataobj)
    
    # Get middle slice
    slice = image_array[image_array.shape[0] // 2, :, :]
    
    # Scale the image such that the maximum pixel value is 255
    # Display the scaled image
    display(Image.fromarray(((slice / np.max(slice)) * 255).astype(np.uint8)))
    
    return 

# Display the middle slice of a 3d array
def display_array(array):
    
    # Get middle slice
    slice = array[array.shape[0] // 2, :, :]
    
    # Scale the image such that the maximum pixel value is 255
    # Display the scaled image
    display(Image.fromarray(((slice / np.max(slice)) * 255).astype(np.uint8)))
    
    return 

# Find files with a specific filename and return a list. Non-recursive 
def list_files_ext(data_path, extensions):

    files = [f for f in os.listdir(data_path) if f.endswith(extensions)]
        
    return files       

# Return the absolute path to all files matching a filename in a directory. Recursive
def list_files_fname(data_path, filename):
    
    matched_files = []
    
    for root, dirs, files in os.walk(data_path):
        
        for file in fnmatch.filter(files, filename):
            
            matched_files.append(os.path.join(root, file))
    
    return matched_files


## Subject class

In [ ]:
class Subject:
    def __init__(self, path):
        
        self.path = path
        
        self.orig_nu = os.path.join(path, "mri/orig_nu.mgz")
        
        self.mask = os.path.join(path, "mri/mask.mgz")
        
        self.aparc = os.path.join(path, "mri/aparc.DKTatlas+aseg.deep.mgz")
        
        aligned_brain = os.path.join(path, "aligned_brain.nii")
        
        self.aligned_brain = aligned_brain if os.path.isfile(aligned_brain) else None
        
        aligned_cropped_brain = os.path.join(path, "aligned_brain_cropped.nii")
        
        self.aligned_cropped_brain = aligned_cropped_brain if os.path.isfile(aligned_cropped_brain) else None
        
        xml_files = glob.glob(os.path.join(path, "*.xml"))
        
        self.xml_path = xml_files[0] if xml_files else None
        
        # Load XML if file exists
        if self.xml_path:
            
            with open(self.xml_path, 'r') as file:
            
                self.xml_df = xmltodict.parse(file.read())
            
        else:
            
            self.xml_df = None

        
        # Manually assign the column headers
        header = ['ColHeaders', 'Index', 'SegId', 'NVoxels', 'Volume_mm3', 'StructName', 'normMean', 'normStdDev', 'normMin', 'normMax', 'normRange']
        
        self.aseg_stats = pd.read_csv(os.path.join(path, 'stats/aseg+DKT.stats'), delimiter='\s+', comment='#', header=None, names=header)

    def display_info(self):
        """Displays basic information about the subject"""
        print(f"Subject File Path: {self.file_path}")
        print(f"DataFrame Head:\n{self.aseg_stats.head()}")
        
# Searches data_path for subject directories and creates an object for each of them
def find_subjects(data_path):
    
    subject_list = []

    for item in os.listdir(data_path):
        
        subject_path = os.path.join(data_path, item)
        
        if os.path.isdir(subject_path):
            
            # MRI directory of subject path (checking validity)
            mri_path = os.path.join(subject_path, 'mri')
            
            # Check for MRI directory
            if os.path.isdir(mri_path):
                
                orig_file = os.path.join(mri_path, 'orig_nu.mgz')
                
                mask_file = os.path.join(mri_path, 'mask.mgz')

                # If both orig.mgz and mask.mgz exist, create object
                if os.path.isfile(orig_file) and os.path.isfile(mask_file):
                    
                    subject_list.append(Subject(subject_path))

    return subject_list

## Image manipulation functions

### Brain extraction

In [4]:
# Performs brain extraction using the orig_nu.mgz and mask.mgz of the subject by multiplying the mask with the image
def extract_brain(orig_file, mask_file):
    
    # Load the image and the brain mask
    image = nibabel.load(orig_file)
    mask = nibabel.load(mask_file)
    
    # Get their image arrays
    image_array = np.asarray(image.dataobj)
    mask_array = np.asarray(mask.dataobj)
    
    # Apply the mask, the mask entries are 1 or 0
    brain_array = image_array * mask_array
    
    return brain_array

### Reference brain (global)

In [ ]:
# NB seems that fastsrufer brain is better
reference_brain_array_mni = extract_brain("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/mni_icbm152_lin_nifti/icbm_avg_152_t1_tal_lin.nii", "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/mni_icbm152_lin_nifti/icbm_avg_152_t1_tal_lin_mask.nii")

display_array(reference_brain_array_mni)

reference_brain_array_fastsurfer = extract_brain("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/mni_icbm152_lin_nifti/fastsurfer-processed/mri/orig_nu.mgz", "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/mni_icbm152_lin_nifti/fastsurfer-processed/mri/mask.mgz")

display_array(reference_brain_array_fastsurfer)


### Affine alignment

In [6]:
# Function for affine registration and saving the transformation
def affine_alignment(subject_list):
        
    # Use ProcessPoolExecutor to run affine alignment in parallel
    with concurrent.futures.ProcessPoolExecutor() as executor:
        
        futures = []
        
        for subject in subject_list:

            futures.append(executor.submit(alignment, subject))
        
        for future in concurrent.futures.as_completed(futures):
            
            display_image(future.result())
            
    
    def alignment(subject):
        
        # Extract brain of subject and convert it to an ANTsPy image
        # The subject's brain is the moving image
        brain_array = extract_brain(subject.orig, subject.mask)
        
        moving_image = ants.from_numpy(brain_array)
        
        # Convert the reference brain to an ANTsPy image
        # The reference brain is already extracted
        fixed_image = ants.from_numpy(reference_brain_array_fastsurfer)
        
        # Perform registration using ANTsPy
        registration = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform='AffineFast')
        
        aligned_brain_array = registration['warpedmovout'].numpy()
        
        # Load the .mat file containing the transformation
        transformation = sio.loadmat(registration['fwdtransforms'][0])
        
        # Save the transformation to a pickle file
        with open(os.path.join(subject.path, 'affine_alignment.pkl'), 'wb') as file:
            pickle.dump(transformation, file)
        
        # Make nibabel image from array
        # Identity matrix as affine transform
        aligned_image = nibabel.Nifti1Image(aligned_brain_array, np.eye(4))
        
        # Save the NiBabel image as a .nii file
        aligned_image_path = os.path.join(subject.path, 'aligned_brain.nii')
        
        nibabel.save(aligned_image, aligned_image_path)
        
        subject.aligned_brain = aligned_image_path
        
        return aligned_image

### Cropping

In [7]:
def crop_subjects(subject_list):
    
    def bounding_box(image_array):
    
        non_zero_indices = np.nonzero(image_array)
        
        min_x, min_y, min_z = np.min(non_zero_indices[0]), np.min(non_zero_indices[1]), np.min(non_zero_indices[2])
        max_x, max_y, max_z = np.max(non_zero_indices[0]), np.max(non_zero_indices[1]), np.max(non_zero_indices[2])
        
        return (min_x, min_y, min_z, max_x, max_y, max_z)
    
    max_bbox = (np.inf, np.inf, np.inf, -np.inf, -np.inf, -np.inf)
    
    # Find maximum bbox
    for subject in subject_list:
    
        image = nibabel.load(os.path.join(subject.path, 'aligned_brain.nii'))
        
        min_x, min_y, min_z, max_x, max_y, max_z = bounding_box(image.get_fdata())
        
        global_min_x, global_min_y, global_min_z, global_max_x, global_max_y, global_max_z = max_bbox
        
        max_bbox = (
            min(global_min_x, min_x),
            min(global_min_y, min_y),
            min(global_min_z, min_z),
            max(global_max_x, max_x),
            max(global_max_y, max_y),
            max(global_max_z, max_z)
        )
    
    # Crop the images
    for subject in subject_list:
        
        image = nibabel.load(os.path.join(subject.path, 'aligned_brain.nii'))
        
        image_array = image.get_fdata()
        
        global_min_x, global_min_y, global_min_z, global_max_x, global_max_y, global_max_z = max_bbox
        
        # Crop the image array using the global bounding box
        cropped_array = image_array[
            int(global_min_x):int(global_max_x),
            int(global_min_y):int(global_max_y),
            int(global_min_z):int(global_max_z)
        ]
        
        # Create a new NiBabel image from the cropped array
        cropped_image = nibabel.Nifti1Image(cropped_array, image.affine)
        
        display_image(cropped_image)
        
        cropped_path = os.path.join(subject.path, 'aligned_brain_cropped.nii')
        
        # Overwrite the original image by saving the cropped image back to the same path
        nibabel.save(cropped_image, cropped_path)
        
        subject.aligned_cropped_brain = cropped_path

## Main code

In [8]:
subject_list = find_subjects("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch disk/full-datasets/adni1-complete-1yr-3t")

In [ ]:
for subject in subject_list:
    
    research_group = subject.xml_df['idaxs']['project']['subject']['researchGroup']
    
    print(research_group)
    
    display_image(nibabel.load(subject.aligned_cropped_brain))


In [ ]:
from nibabel import freesurfer

aparc = nibabel.load("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch disk/preprocess-sample/test-subject/mri/aparc.DKTatlas+aseg.deep.mgz")



print(aparc.header)


annot = nibabel.freesurfer.read_annot("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch disk/preprocess-sample/test-subject/mri/aparc.DKTatlas+aseg.deep.mgz")

Todo:
- Crop images to reduce the size of 3d CNN (could crop to the boundaries of the max brain after registration)
- Load XML disease label and store with image. NB could implement a class that stores various image data
- Start training NN

NB no need for intensity normalisation this is already done by freesurfer/fastsurfer